<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 95%; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">Santander Customer Satisfaction: Basic Preprocessing</span></b>
  </div>
</div>

<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **This notebook performs the initial data preprocessing for the [Santander Customer Satisfaction competition](https://www.kaggle.com/c/santander-customer-satisfaction/overview).**

**The main steps include:**

<div class="alert alert-danger">

**Workflow Overview**
</div>


1. **Import Libraries and Setup**: Load required Python libraries and configure the environment.
2. **Load Data**: Read the raw training and test datasets from CSV files.
3. **Initial Cleaning**: Remove constant columns and duplicate rows from the datasets.
4. **Feature Intersection**: Ensure that both train and test datasets have the same set of features.
5. **Low-Variance Feature Removal**: Identify and drop features with very low variance, as they add little value to modeling.
6. **Missing Value Analysis**: Check for missing values in the cleaned training data.
7. **Correlation Analysis**: Identify highly correlated features that may be redundant.
8. **Save Cleaned Data**: Store the cleaned datasets for further analysis and modeling.


<div class="alert alert-danger">

**Key Functions and Steps**
</div>

- `data_utils.load_data`: Loads CSV data into a pandas DataFrame.
- `data_utils.initial_cleaning`: Removes constant columns and duplicates.
- `data_utils.intersect_features`: Ensures feature consistency between train and test sets.
- `VarianceThreshold`: Used to remove low-variance features.


<div class="alert alert-danger">

**Outputs**
</div>

- Cleaned train and test datasets saved to the `data/processed/` directory.
- Summary statistics and information about features removed or flagged for further analysis.

</div>


---

<div class="alert alert-warning">

**Note:** This notebook is intended as a starting point for feature engineering and model development. Further steps may include advanced feature selection, handling missing values, and building predictive models.
</div>

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Import Libraries and Setup**

</div>

In [1]:
import sys
import os
# Add the parent directory to the Python path
sys.path.append(os.path.abspath('..'))


from classes import Paths
import src.data_utils as data_utils
import pandas as pd
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_columns", None)
import numpy as np
import warnings
warnings.filterwarnings("ignore")


from src.data_utils import setup_logging
setup_logging()

from sklearn.feature_selection import VarianceThreshold

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Load Data**

</div>

In [2]:
paths = Paths()
train_data = data_utils.load_data(paths.get_raw_data_path("train.csv"))
test_data = data_utils.load_data(paths.get_raw_data_path("test.csv"))


display(train_data.head())
print("Train shape: {}".format(train_data.shape))
display(test_data.head())
print("Test shape: {}".format(test_data.shape))

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,ind_var29_0,ind_var29,ind_var30_0,ind_var30,ind_var31_0,ind_var31,ind_var32_cte,ind_var32_0,ind_var32,ind_var33_0,ind_var33,ind_var34_0,ind_var34,ind_var37_cte,ind_var37_0,ind_var37,ind_var39_0,ind_var40_0,ind_var40,ind_var41_0,ind_var41,ind_var39,ind_var44_0,ind_var44,ind_var46_0,ind_var46,num_var1_0,num_var1,num_var4,num_var5_0,num_var5,num_var6_0,num_var6,num_var8_0,num_var8,num_var12_0,num_var12,num_var13_0,num_var13_corto_0,num_var13_corto,num_var13_largo_0,num_var13_largo,num_var13_medio_0,num_var13_medio,num_var13,num_var14_0,num_var14,num_var17_0,num_var17,num_var18_0,num_var18,num_var20_0,num_var20,num_var24_0,num_var24,num_var26_0,num_var26,num_var25_0,num_var25,num_op_var40_hace2,num_op_var40_hace3,num_op_var40_ult1,num_op_var40_ult3,num_op_var41_hace2,num_op_var41_hace3,num_op_var41_ult1,num_op_var41_ult3,num_op_var39_hace2,num_op_var39_hace3,num_op_var39_ult1,num_op_var39_ult3,num_var27_0,num_var28_0,num_var28,num_var27,num_var29_0,num_var29,num_var30_0,num_var30,num_var31_0,num_var31,num_var32_0,num_var32,num_var33_0,num_var33,num_var34_0,num_var34,num_var35,num_var37_med_ult2,num_var37_0,num_var37,num_var39_0,num_var40_0,num_var40,num_var41_0,num_var41,num_var39,num_var42_0,num_var42,num_var44_0,num_var44,num_var46_0,num_var46,saldo_var1,saldo_var5,saldo_var6,saldo_var8,saldo_var12,saldo_var13_corto,saldo_var13_largo,saldo_var13_medio,saldo_var13,saldo_var14,saldo_var17,saldo_var18,saldo_var20,saldo_var24,saldo_var26,saldo_var25,saldo_var28,saldo_var27,saldo_var29,saldo_var30,saldo_var31,saldo_var32,saldo_var33,saldo_var34,saldo_var37,saldo_var40,saldo_var41,saldo_var42,saldo_var44,saldo_var46,var36,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,delta_imp_trasp_var17_out_1y3,delta_imp_trasp_var33_in_1y3,delta_imp_trasp_var33_out_1y3,delta_imp_venta_var44_1y3,delta_num_aport_var13_1y3,delta_num_aport_var17_1y3,delta_num_aport_var33_1y3,delta_num_compra_var44_1y3,delta_num_reemb_var13_1y3,delta_num_reemb_var17_1y3,delta_num_reemb_var33_1y3,delta_num_trasp_var17_in_1y3,delta_num_trasp_var17_out_1y3,delta_num_trasp_var33_in_1y3,delta_num_trasp_var33_out_1y3,delta_num_venta_var44_1y3,imp_amort_var18_hace3,imp_amort_var18_ult1,imp_amort_var34_hace3,imp_amort_var34_ult1,imp_aport_var13_hace3,imp_aport_var13_ult1,imp_aport_var17_hace3,imp_aport_var17_ult1,imp_aport_var33_hace3,imp_aport_var33_ult1,imp_var7_emit_ult1,imp_var7_recib_ult1,imp_compra_var44_hace3,imp_compra_var44_ult1,imp_reemb_var13_hace3,imp_reemb_var13_ult1,imp_reemb_var17_hace3,imp_reemb_var17_ult1,imp_reemb_var33_hace3,imp_reemb_var33_ult1,imp_var43_emit_ult1,imp_trans_var37_ult1,imp_trasp_var17_in_hace3,imp_trasp_var17_in_ult1,imp_trasp_var17_out_hace3,imp_trasp_var17_out_ult1,imp_trasp_var33_in_hace3,imp_trasp_var33_in_ult1,imp_trasp_var33_out_hace3,imp_trasp_var33_out_ult1,imp_venta_var44_hace3,imp_venta_var44_ult1,ind_var7_emit_ult1,ind_var7_recib_ult1,ind_var10_ult1,ind_var10

Train shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,ind_var29_0,ind_var29,ind_var30_0,ind_var30,ind_var31_0,ind_var31,ind_var32_cte,ind_var32_0,ind_var32,ind_var33_0,ind_var33,ind_var34_0,ind_var34,ind_var37_cte,ind_var37_0,ind_var37,ind_var39_0,ind_var40_0,ind_var40,ind_var41_0,ind_var41,ind_var39,ind_var44_0,ind_var44,ind_var46_0,ind_var46,num_var1_0,num_var1,num_var4,num_var5_0,num_var5,num_var6_0,num_var6,num_var8_0,num_var8,num_var12_0,num_var12,num_var13_0,num_var13_corto_0,num_var13_corto,num_var13_largo_0,num_var13_largo,num_var13_medio_0,num_var13_medio,num_var13,num_var14_0,num_var14,num_var17_0,num_var17,num_var18_0,num_var18,num_var20_0,num_var20,num_var24_0,num_var24,num_var26_0,num_var26,num_var25_0,num_var25,num_op_var40_hace2,num_op_var40_hace3,num_op_var40_ult1,num_op_var40_ult3,num_op_var41_hace2,num_op_var41_hace3,num_op_var41_ult1,num_op_var41_ult3,num_op_var39_hace2,num_op_var39_hace3,num_op_var39_ult1,num_op_var39_ult3,num_var27_0,num_var28_0,num_var28,num_var27,num_var29_0,num_var29,num_var30_0,num_var30,num_var31_0,num_var31,num_var32_0,num_var32,num_var33_0,num_var33,num_var34_0,num_var34,num_var35,num_var37_med_ult2,num_var37_0,num_var37,num_var39_0,num_var40_0,num_var40,num_var41_0,num_var41,num_var39,num_var42_0,num_var42,num_var44_0,num_var44,num_var46_0,num_var46,saldo_var1,saldo_var5,saldo_var6,saldo_var8,saldo_var12,saldo_var13_corto,saldo_var13_largo,saldo_var13_medio,saldo_var13,saldo_var14,saldo_var17,saldo_var18,saldo_var20,saldo_var24,saldo_var26,saldo_var25,saldo_var28,saldo_var27,saldo_var29,saldo_var30,saldo_var31,saldo_var32,saldo_var33,saldo_var34,saldo_var37,saldo_var40,saldo_var41,saldo_var42,saldo_var44,saldo_var46,var36,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,delta_imp_trasp_var17_out_1y3,delta_imp_trasp_var33_in_1y3,delta_imp_trasp_var33_out_1y3,delta_imp_venta_var44_1y3,delta_num_aport_var13_1y3,delta_num_aport_var17_1y3,delta_num_aport_var33_1y3,delta_num_compra_var44_1y3,delta_num_reemb_var13_1y3,delta_num_reemb_var17_1y3,delta_num_reemb_var33_1y3,delta_num_trasp_var17_in_1y3,delta_num_trasp_var17_out_1y3,delta_num_trasp_var33_in_1y3,delta_num_trasp_var33_out_1y3,delta_num_venta_var44_1y3,imp_amort_var18_hace3,imp_amort_var18_ult1,imp_amort_var34_hace3,imp_amort_var34_ult1,imp_aport_var13_hace3,imp_aport_var13_ult1,imp_aport_var17_hace3,imp_aport_var17_ult1,imp_aport_var33_hace3,imp_aport_var33_ult1,imp_var7_emit_ult1,imp_var7_recib_ult1,imp_compra_var44_hace3,imp_compra_var44_ult1,imp_reemb_var13_hace3,imp_reemb_var13_ult1,imp_reemb_var17_hace3,imp_reemb_var17_ult1,imp_reemb_var33_hace3,imp_reemb_var33_ult1,imp_var43_emit_ult1,imp_trans_var37_ult1,imp_trasp_var17_in_hace3,imp_trasp_var17_in_ult1,imp_trasp_var17_out_hace3,imp_trasp_var17_out_ult1,imp_trasp_var33_in_hace3,imp_trasp_var33_in_ult1,imp_trasp_var33_out_hace3,imp_trasp_var33_out_ult1,imp_venta_var44_hace3,imp_venta_var44_ult1,ind_var7_emit_ult1,ind_var7_recib_ult1,ind_var10_ult1,ind_var10

Test shape: (75818, 370)


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Initial Cleaning**

</div>

In [3]:
# Initial cleaning and feature intersection
data_utils.initial_cleaning(train_data)

# Some feature values are present in train and absent in test and vice-versa.
y_train = train_data['TARGET']
df_train, df_test = data_utils.intersect_features(train_dataframe=train_data, test_dataframe=test_data)

y_train.head()

Removed constant columns: ['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0', 'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46', 'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3', 'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3', 'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3', 'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3']
Finding common features between train and test datasets.
Finding common features between train and test datasets.


0    0
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [4]:
df_train.shape, df_test.shape

((76020, 370), (75818, 370))

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Feature Intersection**

</div>

In [5]:
print(df_train.shape)
df_train.info()
df_train.describe(include='all')  # Transpose for easier reading

(76020, 370)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 370 entries, num_trasp_var17_out_hace3 to num_meses_var13_medio_ult3
dtypes: float64(111), int64(259)
memory usage: 214.6 MB


,num_trasp_var17_out_hace3,ind_var20,saldo_var20,imp_op_var41_comer_ult3,saldo_var13_largo,num_aport_var13_hace3,num_op_var40_ult1,num_op_var40_efect_ult3,delta_imp_reemb_var17_1y3,num_op_var41_efect_ult3,imp_trasp_var33_out_ult1,var21,num_venta_var44_hace3,imp_compra_var44_ult1,ind_var25,num_var40,num_var45_hace3,ind_var12,num_op_var41_comer_ult1,saldo_medio_var13_medio_hace2,imp_var7_emit_ult1,num_var24,ind_var5,num_aport_var17_ult1,num_var37_0,num_var45_ult1,saldo_var29,ind_var25_0,var15,var3,ind_var5_0,ind_var46,ind_var31,num_var13_medio_0,saldo_medio_var12_hace2,num_compra_var44_hace3,saldo_medio_var13_medio_ult1,num_var14_0,num_var8_0,imp_trasp_var33_in_ult1,num_var29_0,ind_var25_cte,num_var13,ind_var9_ult1,imp_venta_var44_hace3,delta_num_aport_var13_1y3,delta_num_trasp_var33_in_1y3,imp_trasp_var17_out_ult1,num_meses_var17_ult3,num_var43_recib_ult1,saldo_var40,num_var44_0,ind_var24,saldo_medio_var5_hace3,delta_num_trasp_var17_out_1y3,num_op_var39_efect_ult3,delta_imp_trasp_var33_out_1y3,num_reemb_var33_hace3,saldo_medio_var17_hace3,saldo_var6,ind_var32_cte,num_var26_0,num_reemb_var13_hace3,saldo_medio_var33_ult1,ind_var13,num_var6,delta_num_aport_var33_1y3,num_var6_0,num_op_var39_efect_ult1,num_var5_0,imp_reemb_var33_ult1,saldo_var1,ind_var20_0,num_var17,num_med_var22_ult3,ind_var18,num_var27_0,num_var13_medio,saldo_var41,ind_var31_0,imp_trasp_var17_in_hace3,num_meses_var5_ult3,saldo_medio_var29_ult1,imp_op_var40_efect_ult3,num_var7_recib_ult1,saldo_medio_var8_ult3,delta_imp_amort_var34_1y3,num_var46,ind_var34_0,ind_var44,imp_op_var40_efect_ult1,num_var28_0,num_meses_var39_vig_ult3,ind_var1_0,num_var13_corto,num_var1,ind_var6_0,num_aport_var33_ult1,num_op_var40_comer_ult1,imp_reemb_var17_ult1,num_var35,ind_var24_0,ind_var13_0,ind_var39,ind_var14_0,delta_num_trasp_var17_in_1y3,var38,num_var7_emit_ult1,ind_var7_recib_ult1,saldo_medio_var13_corto_hace2,ind_var39_0,num_aport_var33_hace3,imp_venta_var44_ult1,ind_var26_0,num_reemb_var33_ult1,num_var17_0,num_var34_0,num_var20_0,ind_var18_0,delta_num_reemb_var13_1y3,num_reemb_var17_hace3,delta_num_aport_var17_1y3,imp_op_var40_ult1,ind_var26,num_var39,imp_op_var41_comer_ult1,num_var18_0,ind_var29_0,num_op_var41_comer_ult3,ind_var43_emit_ult1,num_var37_med_ult2,imp_trasp_var17_in_ult1,num_op_var41_efect_ult1,saldo_medio_var13_largo_hace2,delta_imp_aport_var17_1y3,num_var22_hace3,saldo_medio_var33_ult3,imp_trasp_var33_out_hace3,num_var5,imp_aport_var17_ult1,num_var41,saldo_medio_var8_ult1,saldo_var33,num_var2_0_ult1,num_var45_ult3,delta_num_reemb_var33_1y3,num_var12_0,ind_var6,saldo_var44,ind_var12_0,saldo_var13_corto,ind_var40_0,num_var45_hace2,ind_var29,ind_var28,imp_compra_var44_hace3,ind_var10_ult1,imp_op_var41_efect_ult3,saldo_medio_var13_largo_ult3,saldo_var37,num_sal_var16_ult1,num_var25_0,saldo_medio_var29_hace2,saldo_medio_var13_corto_ult1,num_meses_var29_ult3,ind_var34,num_op_var40_hace2,num_meses_var13_largo_ult3,delta_imp_venta_var44_1y3,ind_var10cte_ult1,ind_var33,imp_trasp_var17_out_hace3,saldo_var8,imp_aport_var17_hace3,num_op_var39_comer_ult3,ind_var27_0,num_var28,delta_imp_aport_var33_1y3,ind_var2_0,num_var18,delta_imp_reemb_var33_1y3,imp_op_var40_comer_ult3,imp_op_var39_efect_ult3,num_var31,ind_var43_recib_ult1,ID,saldo_var24,ind_var8_0,imp_op_var41_ult1,saldo_medio_var5_hace2,num_var12,saldo_medio_var5_ult1,num_op_var40_ult3,ind_var17,ind_var2,ind_var37_cte,num_aport_var17_hace3,num_var26,saldo_var28,num_reemb_var13_ult1,saldo_var13_medio,num_var4,num_op_var39_hace2,saldo_medio_var13_largo_hace3,imp_sal_var16_ult1,num_var31_0,saldo_medio_var8_hace2,num_var1_0,saldo_medio_var29_hace3,num_trasp_var17_out_ult1,imp_aport_var13_hace3,delta_imp_reemb_var13_1y3,num_var13_largo_0,num_var40_0,saldo_medio_var8_hace3,num_var27,num_var39_0,num_op_var40_efect_ult1,saldo_medio_var13_corto_hace3,imp_trans_var37_ult1,delta_imp_aport_var13_1y3,num_var29,num_op_var39_comer_ult1,saldo_medio_var12_ult3,saldo_var18,num_var30_0,imp_op_var39_comer_ult3,ind_var7_emit_ult1,num_venta_var4

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Low-Variance Feature Removal**

</div>

In [6]:
# Identify constant or near-constant features (low-variance features)
# These add no value and can be dropped.

selector = VarianceThreshold(threshold=0.01)
selector.fit(df_train.select_dtypes(include=['float64', 'int64']))

low_variance_cols = df_train.select_dtypes(include=['float64', 'int64']).columns[~selector.get_support()]
print("There are {} low-variance columns".format(len(low_variance_cols)))
print("Low-variance columns:", low_variance_cols.tolist())


There are 97 low-variance columns
Low-variance columns: ['num_trasp_var17_out_hace3', 'ind_var20', 'num_venta_var44_hace3', 'ind_var46', 'ind_var31', 'num_var13_medio_0', 'num_compra_var44_hace3', 'num_var29_0', 'num_meses_var17_ult3', 'num_reemb_var33_hace3', 'ind_var32_cte', 'num_reemb_var13_hace3', 'num_var6', 'num_var6_0', 'ind_var20_0', 'ind_var18', 'num_var27_0', 'num_var13_medio', 'saldo_var41', 'ind_var31_0', 'num_var46', 'ind_var34_0', 'ind_var44', 'num_var28_0', 'ind_var6_0', 'num_aport_var33_ult1', 'ind_var39', 'num_var7_emit_ult1', 'ind_var7_recib_ult1', 'num_aport_var33_hace3', 'num_reemb_var33_ult1', 'num_var34_0', 'ind_var18_0', 'num_reemb_var17_hace3', 'num_var18_0', 'ind_var29_0', 'imp_trasp_var33_out_hace3', 'num_var41', 'num_var2_0_ult1', 'ind_var6', 'ind_var29', 'ind_var28', 'num_meses_var29_ult3', 'ind_var34', 'ind_var33', 'imp_trasp_var17_out_hace3', 'ind_var27_0', 'num_var28', 'ind_var2_0', 'num_var18', 'ind_var17', 'ind_var2', 'saldo_var28', 'num_reemb_var13_ult

In [7]:
df_train.drop(columns=low_variance_cols.to_list(), inplace=True)
df_train, df_test = data_utils.intersect_features(train_dataframe=df_train, test_dataframe=df_test)

Finding common features between train and test datasets.


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Missing Value Analysis**

</div>

In [8]:
missing = df_train.isnull().mean().sort_values(ascending=False)
print(missing[missing > 0])

Series([], dtype: float64)


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Correlation Analysis**

</div>

In [9]:
corr = df_train.corr(numeric_only=True).abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# Find highly correlated features (e.g., >0.95)
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print("Highly correlated features to drop:", to_drop)

Highly correlated features to drop: ['num_op_var40_ult1', 'num_var4', 'num_var24_0', 'num_op_var39_hace2', 'imp_compra_var44_ult1', 'ind_var25', 'num_var13_largo_0', 'num_var40_0', 'delta_num_reemb_var13_1y3', 'ind_var25_0', 'num_var39_0', 'ind_var26', 'num_var39', 'num_op_var39_comer_ult1', 'saldo_medio_var12_ult3', 'imp_op_var39_comer_ult3', 'saldo_medio_var13_medio_ult1', 'num_var37', 'delta_imp_aport_var17_1y3', 'num_var13', 'delta_num_aport_var13_1y3', 'num_var5', 'delta_num_trasp_var33_out_1y3', 'saldo_var12', 'saldo_var34', 'saldo_var33', 'num_meses_var12_ult3', 'num_op_var41_ult1', 'saldo_var17', 'ind_var24', 'ind_var8', 'imp_op_var39_efect_ult1', 'num_op_var39_efect_ult3', 'delta_imp_trasp_var17_in_1y3', 'delta_imp_trasp_var33_out_1y3', 'imp_amort_var18_ult1', 'saldo_medio_var17_hace3', 'saldo_var6', 'ind_var40_0', 'num_var26_0', 'num_meses_var13_corto_ult3', 'saldo_medio_var33_ult1', 'ind_var10_ult1', 'ind_var13', 'saldo_medio_var13_largo_ult3', 'num_var25_0', 'num_op_var39_e

In [10]:
df_train.drop(columns=to_drop, inplace=True)
df_train, df_test = data_utils.intersect_features(train_dataframe=df_train, test_dataframe=df_test)
print("After dropping highly correlated features, train shape:", df_train.shape)

Finding common features between train and test datasets.
After dropping highly correlated features, train shape: (76020, 184)


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

### **Save Cleaned Data**

</div>

In [11]:
# Save the cleaned data
df_train.to_csv(paths.get_processed_data_path("train_cleaned.csv"), index=False)
df_test.to_csv(paths.get_processed_data_path("test_cleaned.csv"), index=False)


<div class="alert" style="background-color:#FFFFFF; border-left: 8px solid #5ad197; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-success">

### **Conclusion**
</div>

The initial data preprocessing phase has been successfully completed. The following key steps were performed:

- Removed constant and duplicate features to eliminate noise and redundancy.  
- Ensured feature consistency between the training and test sets.  
- Dropped low-variance features that do not contribute significantly to model learning.  
- Verified that there are no missing values in the dataset.  
- Identified and removed highly correlated features to reduce multicollinearity.  

The cleaned datasets have been saved and are now ready for advanced feature engineering, exploratory data analysis (EDA), and model development.

</div>
